## Work
1. 請比較使用 l1, l1_l2 及不同比例下的訓練結果

In [1]:
import os
import keras
import itertools

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.regularizers import l1_l2

"""
建立神經網路
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128], l1_ratio=0.0, l2_ratio=0.0):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1), 
                                   kernel_regularizer=l1_l2(l1=l1_ratio, l2=l2_ratio))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1),
                                   kernel_regularizer=l1_l2(l1=l1_ratio, l2=l2_ratio))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 10
BATCH_SIZE = 256
MOMENTUM = 0.95
L1_EXP = [1e-2, 1e-4, 1e-8, 1e-12, 0.0]
L2_EXP = [1e-2, 1e-4, 1e-8, 1e-12, 0.0]

In [7]:
results = {}
for l1r, l2r in itertools.product(L1_EXP, L2_EXP):
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with L1 = %.6f, L2 = %.6f" % (l1r, l2r))
    model = build_mlp(input_shape=x_train.shape[1:], l1_ratio=l1r, l2_ratio=l2r)
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "exp-l1-%s-l2-%s" % (str(l1r), str(l2r))
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}




Experiment with L1 = 0.010000, L2 = 0.010000


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rul

Experiment with L1 = 0.010000, L2 = 0.000000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [=============

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 4s 85us/step - loss: 18.8393 - acc: 0.2647 - val_loss: 17.3873 - val_acc: 0.3426
Epoch 2/10
50000/50000 [==============================] - 4s 75us/step - loss: 16.1676 - acc: 0.3557 - val_loss: 14.9976 - val_acc: 0.3716
Epoch 3/10
50000/50000 [==============================] - 4s 76us/step - loss: 13.9794 - acc: 0.3783 - val_loss: 12.9939 - val_acc: 0.3841
Epoch 4/10
50000/50000 [==============================] - 4s 75us/step - loss: 12.1272 - acc: 0.3950 - val_loss: 11.2865 - val_acc: 0.3974
Epoch 5/10
50000/50000 [==============================] - 4s 75us/step - loss: 10.5533 - acc: 0.4033 - val_loss: 9.8440 - val_acc: 0.4047
Epoch 6/10
50000/50000 [==============================] - 4s 76us/step - loss: 9.2152 - acc: 0.4103 - val_loss: 8.6116 - val_acc: 0.4068
Epoch 7/10
50000/50000 [==============================] - 4s 76us/step - loss: 8.0749 - acc: 0.4130 - val_loss: 7.5602 

50000/50000 [==============================] - 4s 76us/step - loss: 5.2406 - acc: 0.4472 - val_loss: 5.2065 - val_acc: 0.4425
Epoch 7/10
50000/50000 [==============================] - 4s 76us/step - loss: 5.1462 - acc: 0.4566 - val_loss: 5.1122 - val_acc: 0.4535
Epoch 8/10
50000/50000 [==============================] - 4s 76us/step - loss: 5.0575 - acc: 0.4638 - val_loss: 5.0438 - val_acc: 0.4571
Epoch 9/10
50000/50000 [==============================] - 4s 76us/step - loss: 4.9727 - acc: 0.4724 - val_loss: 4.9518 - val_acc: 0.4663
Epoch 10/10
50000/50000 [==============================] - 4s 76us/step - loss: 4.8896 - acc: 0.4810 - val_loss: 4.8757 - val_acc: 0.4719
Experiment with L1 = 0.000100, L2 = 0.000000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (D

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 4s 86us/step - loss: 2.0224 - acc: 0.2772 - val_loss: 1.8704 - val_acc: 0.3355
Epoch 2/10
50000/50000 [==============================] - 4s 77us/step - loss: 1.8037 - acc: 0.3671 - val_loss: 1.7794 - val_acc: 0.3715
Epoch 3/10
50000/50000 [==============================] - 4s 77us/step - loss: 1.7218 - acc: 0.3956 - val_loss: 1.6905 - val_acc: 0.4050
Epoch 4/10
50000/50000 [==============================] - 4s 77us/step - loss: 1.6644 - acc: 0.4157 - val_loss: 1.6448 - val_acc: 0.4200
Epoch 5/10
50000/50000 [==============================] - 4s 77us/step - loss: 1.6160 - acc: 0.4335 - val_loss: 1.6040 - val_acc: 0.4301
Epoch 6/10
50000/50000 [==============================] - 4s 77us/step - loss: 1.5761 - acc: 0.4477 - val_loss: 1.5697 - val_acc: 0.4474
Epoch 7/10
50000/50000 [==============================] - 4s 77us/step - loss: 1.5407 - acc: 0.4593 - val_loss: 1.5457 - val_acc

50000/50000 [==============================] - 4s 77us/step - loss: 7.7182 - acc: 0.4121 - val_loss: 7.2535 - val_acc: 0.4163
Epoch 7/10
50000/50000 [==============================] - 4s 76us/step - loss: 6.8448 - acc: 0.4188 - val_loss: 6.4554 - val_acc: 0.4146
Epoch 8/10
50000/50000 [==============================] - 4s 79us/step - loss: 6.1008 - acc: 0.4237 - val_loss: 5.7657 - val_acc: 0.4224
Epoch 9/10
50000/50000 [==============================] - 4s 79us/step - loss: 5.4648 - acc: 0.4303 - val_loss: 5.1751 - val_acc: 0.4328
Epoch 10/10
50000/50000 [==============================] - 4s 76us/step - loss: 4.9205 - acc: 0.4349 - val_loss: 4.6771 - val_acc: 0.4275
Experiment with L1 = 0.000000, L2 = 0.000100
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (D

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 4s 76us/step - loss: 2.0401 - acc: 0.2697 - val_loss: 1.8729 - val_acc: 0.3451
Epoch 2/10
50000/50000 [==============================] - 3s 70us/step - loss: 1.8138 - acc: 0.3626 - val_loss: 1.7736 - val_acc: 0.3801
Epoch 3/10
50000/50000 [==============================] - 3s 69us/step - loss: 1.7338 - acc: 0.3925 - val_loss: 1.7104 - val_acc: 0.3930
Epoch 4/10
50000/50000 [==============================] - 3s 69us/step - loss: 1.6728 - acc: 0.4155 - val_loss: 1.6478 - val_acc: 0.4253
Epoch 5/10
50000/50000 [==============================] - 3s 70us/step - loss: 1.6266 - acc: 0.4305 - val_loss: 1.6053 - val_acc: 0.4427
Epoch 6/10
50000/50000 [==============================] - 3s 70us/step - loss: 1.5860 - acc: 0.4436 - val_loss: 1.5786 - val_acc: 0.4434
Epoch 7/10
50000/50000 [==============================] - 3s 70us/step - loss: 1.5536 - acc: 0.4526 - val_loss: 1.5535 - val_acc

50000/50000 [==============================] - 3s 67us/step - loss: 1.5670 - acc: 0.4506 - val_loss: 1.5619 - val_acc: 0.4476
Epoch 7/10
50000/50000 [==============================] - 3s 67us/step - loss: 1.5313 - acc: 0.4641 - val_loss: 1.5598 - val_acc: 0.4468
Epoch 8/10
50000/50000 [==============================] - 3s 67us/step - loss: 1.5014 - acc: 0.4730 - val_loss: 1.5193 - val_acc: 0.4649
Epoch 9/10
50000/50000 [==============================] - 3s 67us/step - loss: 1.4732 - acc: 0.4836 - val_loss: 1.5077 - val_acc: 0.4685
Epoch 10/10
50000/50000 [==============================] - 3s 67us/step - loss: 1.4484 - acc: 0.4928 - val_loss: 1.4733 - val_acc: 0.4779
Experiment with L1 = 0.000000, L2 = 0.000000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (D

In [14]:
import matplotlib.pyplot as plt
%matplotlib inline
color_bar = ["r", "g", "b", "y", "m", "k"]

len(results[cond]['train-loss'])


for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()

dict_keys(['exp-l1-0.01-l2-0.01', 'exp-l1-0.01-l2-0.0001', 'exp-l1-0.01-l2-1e-08', 'exp-l1-0.01-l2-1e-12', 'exp-l1-0.01-l2-0.0', 'exp-l1-0.0001-l2-0.01', 'exp-l1-0.0001-l2-0.0001', 'exp-l1-0.0001-l2-1e-08', 'exp-l1-0.0001-l2-1e-12', 'exp-l1-0.0001-l2-0.0', 'exp-l1-1e-08-l2-0.01', 'exp-l1-1e-08-l2-0.0001', 'exp-l1-1e-08-l2-1e-08', 'exp-l1-1e-08-l2-1e-12', 'exp-l1-1e-08-l2-0.0', 'exp-l1-1e-12-l2-0.01', 'exp-l1-1e-12-l2-0.0001', 'exp-l1-1e-12-l2-1e-08', 'exp-l1-1e-12-l2-1e-12', 'exp-l1-1e-12-l2-0.0', 'exp-l1-0.0-l2-0.01', 'exp-l1-0.0-l2-0.0001', 'exp-l1-0.0-l2-1e-08', 'exp-l1-0.0-l2-1e-12', 'exp-l1-0.0-l2-0.0'])